In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertForMaskedLM
import tensorflow as tf
import os
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')
model = TFBertForMaskedLM.from_pretrained('/content/gdrive/MyDrive/Bert')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at /content/gdrive/MyDrive/Bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
model.summary()


Model: "tf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108891648 
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  24459834  
                                                                 
Total params: 109,514,298
Trainable params: 109,514,298
Non-trainable params: 0
_________________________________________________________________


In [ ]:
with open('/content/gdrive/MyDrive/train.txt','r') as f:
    file = f.read().split("\n")
text_lst = []
for line in file:
      text_lst.append(line)

with open('/content/gdrive/MyDrive/test.txt','r') as f:
    file = f.read().split("\n")
text_lst_test = []
for line in file:
      text_lst_test.append(line)

In [ ]:
text_lst

['two young , white males are outside near many bushes .',
 'several men in hard hats are operating a giant pulley system .',
 'a little girl climbing into a wooden playhouse .',
 'a man in a blue shirt is standing on a ladder cleaning a window .',
 'two men are at the stove preparing food .',
 'a man in green holds a guitar while the other man observes his shirt .',
 'a man is smiling at a stuffed lion',
 'a trendy girl talking on her cellphone while gliding slowly down the street .',
 'a woman with a large purse is walking by a gate .',
 'boys dancing on poles in the middle of the night .',
 'a ballet class of five girls jumping in sequence .',
 'four guys three wearing hats one not are jumping at the top of a staircase .',
 'a black dog and a spotted dog are fighting',
 'a man in a neon green and orange uniform is driving on a green tractor .',
 'several women wait outside in a city .',
 'a lady in a black top with glasses is sprinkling powdered sugar on a bundt cake .',
 'a little 

In [ ]:
inputs = tokenizer(text_lst,max_length=50,truncation=True,padding='max_length',return_tensors='tf')
inputs_test = tokenizer(text_lst_test,max_length=50,truncation=True,padding='max_length',return_tensors='tf')

In [ ]:
inputs['labels'] = inputs['input_ids']
inputs_test['labels'] = inputs_test['input_ids']
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
inp_ids = []
lbs = []
idx = 0
for inp in inputs.input_ids.numpy():
    actual_tokens = list(set(range(50)) -
                         set(np.where((inp == 101) | (inp == 102)
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens),
                                     size=num_of_token_to_mask,
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
inp_ids = tf.convert_to_tensor(inp_ids)
inputs['input_ids'] = inp_ids



In [ ]:
inp_ids_test = []
lbs_test = []
idx_test = 0

mask_loc = []
for inp in inputs_test.input_ids.numpy():
    actual_tokens = list(set(range(50)) -
                         set(np.where((inp == 101) | (inp == 102)
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens),
                                     size=num_of_token_to_mask,
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids_test.append(inp)
    idx_test += 1
    mask_loc.append(token_to_mask)
inp_ids_test = tf.convert_to_tensor(inp_ids_test)
inputs_test['input_ids'] = inp_ids_test

In [ ]:
inp_ids_test

TensorShape([96, 50])

In [ ]:
inp_ids

<tf.Tensor: shape=(40883, 50), dtype=int32, numpy=
array([[ 101, 2048, 2402, ...,    0,    0,    0],
       [ 101,  103, 2273, ...,    0,    0,    0],
       [ 101, 1037, 2210, ...,    0,    0,    0],
       ...,
       [ 101,   53, 9750, ...,    0,    0,    0],
       [ 101, 1037, 2158, ...,    0,    0,    0],
       [ 101,  102,    0, ...,    0,    0,    0]], dtype=int32)>

In [ ]:
inp_ids_test

<tf.Tensor: shape=(96, 50), dtype=int32, numpy=
array([[ 101, 1037, 2158, ...,    0,    0,    0],
       [ 101, 1037, 3731, ...,    0,    0,    0],
       [ 101, 1037, 2611, ...,    0,    0,    0],
       ...,
       [ 101, 1037, 3287, ...,    0,    0,    0],
       [ 101,  103, 3080, ...,    0,    0,    0],
       [ 101,  102,    0, ...,    0,    0,    0]], dtype=int32)>

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))



In [ ]:
model.fit([inputs.input_ids,inputs.attention_mask],inputs.labels,verbose=1,batch_size=16,epochs=1)

 404/1813 [=====>........................] - ETA: 5:10:44 - loss: 0.0701

In [ ]:
model.save_pretrained("/content/gdrive/MyDrive/Bert", from_pt=True)

In [ ]:
#model.eval()
out = model(inputs_test).logits.numpy()

In [ ]:
mask_loc_test = []

#cnt0 = 0

for i in inputs_test.input_ids:
    cnt1 = 0
    mask = []
    for j in i:
        if j == 103:
           mask.append(cnt1)
        cnt1 += 1
    mask_loc_test.append(mask)

#mask_loc = np.where(inputs_test.input_ids.numpy() == 103)#.tolist()


In [ ]:
inputs_test.input_ids[0]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 101, 1037, 2158, 1999, 2019, 4589, 6045,  103, 2012, 2242, 1012,
        102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)>

In [ ]:
inputs_test.input_ids

<tf.Tensor: shape=(96, 100), dtype=int32, numpy=
array([[ 101, 1037, 2158, ...,    0,    0,    0],
       [ 101, 1037, 3731, ...,    0,    0,    0],
       [ 101, 1037, 2611, ...,    0,    0,    0],
       ...,
       [ 101,  103, 3287, ...,    0,    0,    0],
       [ 101, 2019,  103, ...,    0,    0,    0],
       [ 101,  102,    0, ...,    0,    0,    0]], dtype=int32)>

In [ ]:
predicted_tokens = []
truth_tokens = []

for i in range(len(out)):
    predicted = []
    truth = []
    for j in mask_loc_test[i]:
        predicted.append(np.argmax(out[i][j]))
        truth.append(inputs_test.labels[i][j])
    predicted_tokens.append(predicted)
    truth_tokens.append(truth)


In [ ]:
tokenizer.decode(predicted_tokens[0])

'.'

In [ ]:
tokenizer.decode(np.argmax(out[1],axis = -1))

'[CLS] a boston terrier is running on lush green grass in front of a white fence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
tokenizer.decode(inputs_test.labels[1])

'[CLS] a boston terrier is running on lush green grass in front of a white fence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
tokenizer.decode(truth_tokens[0])

'.'

In [ ]:
cnt = 0
sum = 0
for i in range(len(predicted_tokens)):
  for j in range(len(predicted_tokens[i])):
    sum += 1
    if predicted_tokens[i][j] == truth_tokens[i][j]:
      cnt += 1

In [ ]:
cnt/sum

0.5807799442896936